In [1]:
import elasticsearch
import elasticsearch.helpers
import json

In [4]:
api_key = "SFBlOWtZd0I1VkFjY09Zd3NzbFc6blg4bXNiREZSMHV2Y040TWxnWXVBZw=="
es_host = 'http://localhost:9200'
es = elasticsearch.Elasticsearch(api_key=api_key,cloud_id="Laptops:ZXVyb3BlLXdlc3Q0LmdjcC5lbGFzdGljLWNsb3VkLmNvbSRiNGMwYjBlMDVkMDc0NDJmYjQwNDhkZDljNTFiNTNhZiRjNTk2MmQ5ZDFkZWY0ODY0YThmMmM0M2NhOTRjYThmMg==")

In [5]:
es.info()

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': 'b4c0b0e05d07442fb4048dd9c51b53af', 'cluster_uuid': 'g7JXyotbTKC7bNfne3037w', 'version': {'number': '8.11.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '64cf052f3b56b1fd4449f5454cb88aca7e739d9a', 'build_date': '2023-12-08T11:33:53.634979452Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
index_name = 'laptops'

def read_documents(file_name):
    """
    Returns a generator of documents to be indexed by elastic, read from file_name
    """
    with open(file_name, 'r') as json_file:
        data = json.load(json_file)
        for laptop in data:
            doc = {"_index": index_name, "_source": laptop}
            yield doc

def create_index(es, index_name, body={}):
    # delete index when it already exists
    es.indices.delete(index=index_name, ignore=[400, 404])
    # create the index 
    es.indices.create(index=index_name, body=body)
                
def index_documents(es, collection_file_name, index_name, body={}):
    create_index(es, index_name, body)
    # bulk index the documents from file_name
    return elasticsearch.helpers.bulk(
        es, 
        read_documents(collection_file_name),
        index=index_name,
        chunk_size=2000,
        request_timeout=30
    )

In [7]:
body = {} # no indexing options (leave default)
index_documents(es, 'data/6-laptops-dataset.json', index_name, body)

/var/folders/d_/39m2_j3d7l5859lht1tjp1_h0000gn/T/ipykernel_36938/468895564.py:15: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400, 404])
/var/folders/d_/39m2_j3d7l5859lht1tjp1_h0000gn/T/ipykernel_36938/468895564.py:22: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  return elasticsearch.helpers.bulk(


(4835, [])

In [8]:
count_molecule_query = {
    "query": {
        "simple_query_string": {
            "query": "mac os",
        }
    }
}

In [9]:
term = 'mac os'
body = {"track_total_hits": True, "query": {"query_string": 
                                            {"query": term, 
                                             "default_operator":"AND", 
                                             "auto_generate_synonyms_phrase_query": True }}}
result = es.search(index='laptops', body=body)

In [10]:
result["hits"]["hits"]

[{'_index': 'laptops',
  '_id': '70q-kYwBs1nhOBfiWwH2',
  '_score': 12.182031,
  '_ignored': ['reviews.reviewText.keyword'],
  '_source': {'source': 'Amazon',
   'imageURLs': ['https://m.media-amazon.com/images/I/81VbWDN53oL._AC_SX355_.jpg',
    'https://m.media-amazon.com/images/I/81VbWDN53oL._AC_SX450_.jpg',
    'https://m.media-amazon.com/images/I/81VbWDN53oL._AC_SX425_.jpg',
    'https://m.media-amazon.com/images/I/81VbWDN53oL._AC_SX466_.jpg',
    'https://m.media-amazon.com/images/I/81VbWDN53oL._AC_SX522_.jpg',
    'https://m.media-amazon.com/images/I/81VbWDN53oL._AC_SX569_.jpg',
    'https://m.media-amazon.com/images/I/81VbWDN53oL._AC_SX679_.jpg'],
   'productURL': 'https://www.amazon.com/Apple-MacBook-ME294LL-15-4-Inch-Refurbished/dp/B079C8TPRX',
   'reviewURL': 'https://www.amazon.com/Apple-MacBook-ME294LL-15-4-Inch-Refurbished/product-reviews/B079C8TPRX/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&amp;reviewerType=all_reviews',
   'sku': 'B079C8TPRX',
   'brand': 'Apple',
   'model': '

In [11]:
lucene_query = 'brand:Dell'

# operatingSystem:Linux AND systemMemoryRam:16gb AND totalStorageCapacity:500gb AND screenSize:16 AND batteryLife:long AND productWeight:[1 TO 100]
test_query = "systemMemoryRam:16~ AND totalStorageCapacity:500gb~"

# Define the search query
search_body = {
    "query": {
        "query_string": {
            "query": test_query,
            "default_field": "brand",  # Specify the default field to search
        }
    }
}

# Perform the search
result = es.search(index=index_name, body=search_body)

# Print the results
for hit in result['hits']['hits']:
    print(hit['_source'])

In [13]:
count_query = {
    "query": {
        "match_all": {}
    }
}

document_count = es.count(index=index_name, body=count_query)

num_documents = document_count["count"]

print(f"Number of documents in index '{index_name}': {num_documents}")

Number of documents in index 'laptops': 4835
